# Aula 7 - DPR

[Unicamp - IA368DD: Deep Learning aplicado a sistemas de busca.](https://www.cpg.feec.unicamp.br/cpg/lista/caderno_horario_show.php?id=1779)

Autor: Marcus Vinícius Borela de Castro

[Repositório no github](https://github.com/marcusborela/deep_learning_em_buscas_unicamp)


## Enunciado do Exercício

Fazer o finetuning de um buscador denso

Usar como treino o dataset "tiny" do MS MARCO
https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv

Avaliar o modelo no TREC-COVID, e comparar os resultados com o BM25 e doc2query

Comparar busca "exaustiva" (semelhança do vetor query com todos os vetores do corpus) com a busca aproximada (Approximate Nearest Neighbor - ANN)

Para a busca aproximada, usar os algoritmos existentes na biblioteca sentence-transformers (ex: hnswlib) OU implemente um você mesmo (Bonus!)

Dicas:

    Usar a média dos vetores da última camada (conhecido como mean pooling) do transformer para representar queries e passagens; Alternativamente, usar apenas o vetor do [CLS] da última cada.
    Tente inicialmente uma loss fácil de implementar, como a entropia-cruzada
    Começar o treino a partir do microsoft/MiniLM-L12-H384-uncased
    Avaliar o pipeline usando um modelo já bem treinado: sentence-transformers/all-mpnet-base-v2
    Comparar resultados usando semelhança de cosseno e produto escalar como funções de similaridade
    Para checar se seu codigo de avaliação está correto, comparar o seu desempenho com o do modelo já treinado no MS MARCO:   https://huggingface.co/sentence-transformers/all-MiniLM-L12-v2; O nDCG@10 no TREC-COVID deve ser ~0.47
    Usar a biblioteca do sentence-transformers para avaliar o modelo


## Fase

Utilizando DPR treinado

# Organizando o ambiente

In [1]:
import pickle

In [2]:
import math

In [3]:
import os

In [4]:
import gzip

In [5]:
import json

In [6]:
import pandas as pd

In [7]:
import faiss

In [8]:
from tqdm import tqdm

In [9]:
from transformers import  BatchEncoding

/home/borela/miniconda3/envs/treinapython39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
from transformers import AutoTokenizer, AutoModel
import torch

In [11]:
from evaluate import load

In [12]:
DIRETORIO_TRABALHO = '/home/borela/fontes/deep_learning_em_buscas_unicamp/local/dpr'

In [13]:
if os.path.exists(DIRETORIO_TRABALHO):
    print('pasta já existia!')
else:
    os.makedirs(DIRETORIO_TRABALHO)
    print('pasta criada!')


pasta já existia!


In [329]:
DIRETORIO_RUN = f"{DIRETORIO_TRABALHO}/runs"
CAMINHO_RUN = f"{DIRETORIO_RUN}/run-trec-covid-bm25.txt"
CAMINHO_RUN_BUSCA_APROXIMADA = f"{DIRETORIO_RUN}/run-trec-covid-bm25-aproximada.txt"

In [15]:
if os.path.exists(DIRETORIO_RUN):
    print('pasta já existia!')
else:
    os.makedirs(DIRETORIO_RUN)
    print('pasta criada!')


pasta já existia!


In [16]:
os.environ['JVM_PATH'] = '/usr/lib/jvm/java-11-openjdk-amd64/lib/server/libjvm.so'
os.environ['JAVA_HOME'] = '/usr/lib/jvm/java-11-openjdk-amd64'

In [17]:
from pyserini.search.lucene import LuceneSearcher

In [18]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  

In [19]:
from psutil import virtual_memory

In [20]:
def mostra_memoria(lista_mem=['cpu']):
  """
  Esta função exibe informações de memória da CPU e/ou GPU, conforme parâmetros fornecidos.

  Parâmetros:
  -----------
  lista_mem : list, opcional
      Lista com strings 'cpu' e/ou 'gpu'. 
      'cpu' - exibe informações de memória da CPU.
      'gpu' - exibe informações de memória da GPU (se disponível).
      O valor padrão é ['cpu'].

  Saída:
  -------
  A função não retorna nada, apenas exibe as informações na tela.

  Exemplo de uso:
  ---------------
  Para exibir informações de memória da CPU:
      mostra_memoria(['cpu'])

  Para exibir informações de memória da CPU e GPU:
      mostra_memoria(['cpu', 'gpu'])
  
  Autor: Marcus Vinícius Borela de Castro

  """  
  if 'cpu' in lista_mem:
    vm = virtual_memory()
    ram={}
    ram['total']=round(vm.total / 1e9,2)
    ram['available']=round(virtual_memory().available / 1e9,2)
    # ram['percent']=round(virtual_memory().percent / 1e9,2)
    ram['used']=round(virtual_memory().used / 1e9,2)
    ram['free']=round(virtual_memory().free / 1e9,2)
    ram['active']=round(virtual_memory().active / 1e9,2)
    ram['inactive']=round(virtual_memory().inactive / 1e9,2)
    ram['buffers']=round(virtual_memory().buffers / 1e9,2)
    ram['cached']=round(virtual_memory().cached/1e9 ,2)
    print(f"Your runtime RAM in gb: \n total {ram['total']}\n available {ram['available']}\n used {ram['used']}\n free {ram['free']}\n cached {ram['cached']}\n buffers {ram['buffers']}")
    print('/nGPU')
    gpu_info = !nvidia-smi
  if 'gpu' in lista_mem:
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
      print('Not connected to a GPU')
    else:
      print(gpu_info)


In [21]:
mostra_memoria(['cpu','gpu'])

Your runtime RAM in gb: 
 total 67.35
 available 61.7
 used 4.69
 free 59.78
 cached 2.61
 buffers 0.27
/nGPU
Wed Apr 19 11:51:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.39.01    Driver Version: 510.39.01    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:02:00.0 Off |                  N/A |
|  0%   45C    P8    33W / 370W |     60MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+--

## Fixando as seeds

In [22]:
import random
import torch
import numpy as np

In [23]:
def inicializa_seed(num_semente:int=123):
  """
  Inicializa as sementes para garantir a reprodutibilidade dos resultados do modelo.
  Essa é uma prática recomendada, já que a geração de números aleatórios pode influenciar os resultados do modelo.
  Além disso, a função também configura as sementes da GPU para garantir a reprodutibilidade quando se utiliza aceleração por GPU. 
  
  Args:
      num_semente (int): número da semente a ser utilizada para inicializar as sementes das bibliotecas.
  
  References:
      http://nlp.seas.harvard.edu/2018/04/03/attention.html
      https://github.com/CyberZHG/torch-multi-head-attention/blob/master/torch_multi_head_attention/multi_head_attention.py#L15
  """
  # Define as sementes das bibliotecas random, numpy e pytorch
  random.seed(num_semente)
  np.random.seed(num_semente)
  torch.manual_seed(num_semente)
  
  # Define as sementes da GPU
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

  #torch.cuda.manual_seed(num_semente)
  #Cuda algorithms
  #torch.backends.cudnn.deterministic = True


In [24]:
num_semente=123
inicializa_seed(num_semente)

## Preparando para debug e display

In [25]:
import pandas as pd

In [26]:
#!pip install transformers -q

In [27]:
import transformers

In [28]:
# https://zohaib.me/debugging-in-google-collab-notebook/
# !pip install -Uqq ipdb
import ipdb
# %pdb off # desativa debug em exceção
# %pdb on  # ativa debug em exceção
# ipdb.set_trace(context=8)  para execução nesse ponto

In [29]:
def config_display():
  """
  Esta função configura as opções de display do Pandas.
  """

  # Configurando formato saída Pandas
  # define o número máximo de colunas que serão exibidas
  pd.options.display.max_columns = None

  # define a largura máxima de uma linha
  pd.options.display.width = 1000

  # define o número máximo de linhas que serão exibidas
  pd.options.display.max_rows = 100

  # define o número máximo de caracteres por coluna
  pd.options.display.max_colwidth = 50

  # se deve exibir o número de linhas e colunas de um DataFrame.
  pd.options.display.show_dimensions = True

  # número de dígitos após a vírgula decimal a serem exibidos para floats.
  pd.options.display.precision = 7


In [30]:
def config_debug():
  """
  Esta função configura as opções de debug do PyTorch e dos pacotes
  transformers e datasets.
  """

  # Define opções de impressão de tensores para o modo científico
  torch.set_printoptions(sci_mode=True) 
  """
    Significa que valores muito grandes ou muito pequenos são mostrados em notação científica.
    Por exemplo, em vez de imprimir o número 0.0000012345 como 0.0000012345, 
    ele seria impresso como 1.2345e-06. Isso é útil em situações em que os valores dos tensores 
    envolvidos nas operações são muito grandes ou pequenos, e a notação científica permite 
    uma melhor compreensão dos números envolvidos.  
  """

  # Habilita detecção de anomalias no autograd do PyTorch
  torch.autograd.set_detect_anomaly(True)
  """
    Permite identificar operações que podem causar problemas de estabilidade numérica, 
    como gradientes explodindo ou desaparecendo. Quando essa opção é ativada, 
    o PyTorch verifica se há operações que geram valores NaN ou infinitos nos tensores 
    envolvidos no cálculo do gradiente. Se for detectado um valor anômalo, o PyTorch 
    interrompe a execução e gera uma exceção, permitindo que o erro seja corrigido 
    antes que se torne um problema maior.

    É importante notar que a detecção de anomalias pode ter um impacto significativo 
    no desempenho, especialmente em modelos grandes e complexos. Por esse motivo,
    ela deve ser usada com cautela e apenas para depuração.
  """

  # Configura variável de ambiente para habilitar a execução síncrona (bloqueante) das chamadas da API do CUDA.
  os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
  """
    o Python aguarda o término da execução de uma chamada da API do CUDA antes de executar a próxima chamada. 
    Isso é útil para depurar erros no código que envolve operações na GPU, pois permite que o erro seja capturado 
    no momento em que ocorre, e não depois de uma sequência de operações que pode tornar a origem do erro mais difícil de determinar.
    No entanto, é importante lembrar que esse modo de execução é significativamente mais lento do que a execução assíncrona, 
    que é o comportamento padrão do CUDA. Por isso, é recomendado utilizar esse comando apenas em situações de depuração 
    e removê-lo após a solução do problema.
  """

  # Define o nível de verbosity do pacote transformers para info
  # transformers.utils.logging.set_verbosity_info() 
  
  
  """
    Define o nível de detalhamento das mensagens de log geradas pela biblioteca Hugging Face Transformers 
    para o nível info. Isso significa que a biblioteca irá imprimir mensagens de log informativas sobre
    o andamento da execução, tais como tempo de execução, tamanho de batches, etc.

    Essas informações podem ser úteis para entender o que está acontecendo durante a execução da tarefa 
    e auxiliar no processo de debug. É importante notar que, em alguns casos, a quantidade de informações
    geradas pode ser muito grande, o que pode afetar o desempenho do sistema e dificultar a visualização
    das informações relevantes. Por isso, é importante ajustar o nível de detalhamento de acordo com a 
    necessidade de cada tarefa.
  
    Caso queira reduzir a quantidade de mensagens, comentar a linha acima e 
      descomentar as duas linhas abaixo, para definir o nível de verbosity como error ou warning
  
    transformers.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_warning()
  """


  # Define o modo verbose do xmode, que é utilizado no debug
  # %xmode Verbose 

  """
    Comando usado no Jupyter Notebook para controlar o modo de exibição das informações de exceções.
    O modo verbose é um modo detalhado que exibe informações adicionais ao imprimir as exceções.
    Ele inclui as informações de pilha de chamadas completa e valores de variáveis locais e globais 
    no momento da exceção. Isso pode ser útil para depurar e encontrar a causa de exceções em seu código.
    Ao usar %xmode Verbose, as informações de exceção serão impressas com mais detalhes e informações adicionais serão incluídas.

    Caso queira desabilitar o modo verbose e utilizar o modo plain, 
    comentar a linha acima e descomentar a linha abaixo:
    %xmode Plain
  """

  """
    Dica:
    1.  pdb (Python Debugger)
      Quando ocorre uma exceção em uma parte do código, o programa para a execução e exibe uma mensagem de erro 
      com informações sobre a exceção, como a linha do código em que ocorreu o erro e o tipo da exceção.

      Se você estiver depurando o código e quiser examinar o estado das variáveis ​​e executar outras operações 
      no momento em que a exceção ocorreu, pode usar o pdb (Python Debugger). Para isso, é preciso colocar o comando %debug 
      logo após ocorrer a exceção. Isso fará com que o programa pare na linha em que ocorreu a exceção e abra o pdb,
      permitindo que você explore o estado das variáveis, examine a pilha de chamadas e execute outras operações para depurar o código.


    2. ipdb
      O ipdb é um depurador interativo para o Python que oferece recursos mais avançados do que o pdb,
      incluindo a capacidade de navegar pelo código fonte enquanto depura.
      
      Você pode começar a depurar seu código inserindo o comando ipdb.set_trace() em qualquer lugar do 
      seu código onde deseja pausar a execução e começar a depurar. Quando a execução chegar nessa linha, 
      o depurador entrará em ação, permitindo que você examine o estado atual do seu programa e execute 
      comandos para investigar o comportamento.

      Durante a depuração, você pode usar comandos:
        next (para executar a próxima linha de código), 
        step (para entrar em uma função chamada na próxima linha de código) 
        continue (para continuar a execução normalmente até o próximo ponto de interrupção).

      Ao contrário do pdb, o ipdb é um depurador interativo que permite navegar pelo código fonte em que
      está trabalhando enquanto depura, permitindo que você inspecione variáveis, defina pontos de interrupção
      adicionais e até mesmo execute expressões Python no contexto do seu programa.
  """


In [31]:
config_display()

In [32]:
config_debug()

# Baixando o dataset para avaliação (trec-covid)

## Queries

In [33]:
from pyserini.search import get_topics

In [34]:
topics = get_topics('covid-round5')
print(f'{len(topics)} queries total')

50 queries total


In [35]:
topics[50]

{'question': 'what is known about an mRNA vaccine for the SARS-CoV-2 virus?',
 'query': 'mRNA vaccine coronavirus',
 'narrative': 'Looking for studies specifically focusing on mRNA vaccines for COVID-19, including how mRNA vaccines work, why they are promising, and any results from actual clinical studies.'}

## Relevância (qrel) de teste

In [36]:
if not os.path.exists(f'{DIRETORIO_TRABALHO}/test.tsv'):
    !wget https://huggingface.co/datasets/BeIR/trec-covid-qrels/raw/main/test.tsv
    !mv test.tsv {DIRETORIO_TRABALHO}/

In [37]:
qrel = pd.read_csv(f"{DIRETORIO_TRABALHO}/test.tsv", sep="\t", header=None, 
                   skiprows=1, names=["query", "docid", "rel"])

In [38]:
qrel.head()

,query,docid,rel
0,1,005b2j4b,2
1,1,00fmeepz,1
2,1,g7dhmyyo,2
3,1,0194oljo,1
4,1,021q9884,1


In [39]:
qrel.head()

,query,docid,rel
0,1,005b2j4b,2
1,1,00fmeepz,1
2,1,g7dhmyyo,2
3,1,0194oljo,1
4,1,021q9884,1


In [40]:
qrel["q0"] = "q0"
qrel_dict = qrel.to_dict(orient="list")

In [41]:
qrel_dict['query'][0], qrel_dict['docid'][0], qrel_dict['rel'][0]

(1, '005b2j4b', 2)

## Documentos a serem indexados

In [42]:
if not os.path.exists(f"{DIRETORIO_TRABALHO}/corpus.jsonl.gz"):
    !wget https://huggingface.co/datasets/BeIR/trec-covid/resolve/main/corpus.jsonl.gz
    !mv corpus.jsonl.gz {DIRETORIO_TRABALHO}
    print('Baixado')
else:
    print('Já existia a pasta')

Já existia a pasta


In [43]:
# Descompacte o arquivo para a memória
with gzip.open(f'{DIRETORIO_TRABALHO}/corpus.jsonl.gz', 'rt') as f:
    # Leia o conteúdo do arquivo descompactado
    corpus = [json.loads(line) for line in f]

In [44]:
# Exiba os dados carregados
print(f"{type(corpus)} len(corpus): {len(corpus)} corpus[0] {corpus[0]}" )

<class 'list'> len(corpus): 171332 corpus[0] {'_id': 'ug7v899j', 'title': 'Clinical features of culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia', 'text': 'OBJECTIVE: This retrospective chart review describes the epidemiology and clinical features of 40 patients with culture-proven Mycoplasma pneumoniae infections at King Abdulaziz University Hospital, Jeddah, Saudi Arabia. METHODS: Patients with positive M. pneumoniae cultures from respiratory specimens from January 1997 through December 1998 were identified through the Microbiology records. Charts of patients were reviewed. RESULTS: 40 patients were identified, 33 (82.5%) of whom required admission. Most infections (92.5%) were community-acquired. The infection affected all age groups but was most common in infants (32.5%) and pre-school children (22.5%). It occurred year-round but was most common in the fall (35%) and spring (30%). More than three-quarters of patients (77.5%

In [45]:
corpus[0].keys()

dict_keys(['_id', 'title', 'text', 'metadata'])

In [83]:
lista_doc_id_passage = [doc['_id'] for doc in corpus]
print(f"len(lista_doc_id_passage) {len(lista_doc_id_passage)}")

len(lista_doc_id_passage) 171332


# Funções auxiliares

In [281]:
def gera_embeddings(parm_model:AutoModel , parm_tokenizer:AutoTokenizer, parm_sentences:list, parm_tipo_resumo:str='cls'):
    """
    Função para gerar embeddings de sentenças usando um modelo pré-treinado.

    Args:
        parm_model (AutoModel): Modelo pré-treinado para geração de embeddings.
        parm_tokenizer (AutoTokenizer): Tokenizer associado ao modelo pré-treinado.
        parm_sentences (list): Lista de sentenças para as quais os embeddings serão gerados.
        parm_tipo_resumo (str, opcional): Tipo de resumo a ser aplicado nas sentenças. Pode ser 'cls' para usar o token [CLS]
            ou 'mean' para usar a média das embeddings dos tokens. O padrão é 'cls'.

    Returns:
        embeddings (torch.Tensor): Embeddings gerados para as sentenças, como um tensor do PyTorch.
    """

    # Tokenize sentences
    encoded_input = parm_tokenizer(parm_sentences, padding=True, truncation=True, max_length=512, return_tensors='pt')
    # print('após tokenizer',encoded_input)

    # Executa o tokenizador nas sentenças fornecidas, com opções de padding, truncation e retorno como tensores do PyTorch

    # print(f"padded_batch {padded_batch}")
    encoded_input = BatchEncoding(encoded_input)
    # print('após BatchEncoding',encoded_input)
    # Move os dados para o dispositivo especificado (CPU ou GPU)
    encoded_input = {key: value.to(device) for key, value in encoded_input.items()}

    with torch.no_grad():
        # Desativa o cálculo de gradientes para economizar memória e acelerar a inferência
        model_output = parm_model(**encoded_input)
        # Passa os inputs tokenizados para o modelo e obtém a saída do modelo
    if parm_tipo_resumo == 'cls':
        # Se o tipo de resumo for 'cls', retorna o embedding do token [CLS]
        embeddings = model_output.last_hidden_state[:, 0, :]
        # Seleciona o embedding do token [CLS], que está na primeira posição do output do modelo
    elif parm_tipo_resumo == 'pooler':
        embeddings = model_output.pooler_output
    else:
        # Se o tipo de resumo não for válido, levanta uma exceção
        raise Exception(f"parm_tipo_resumo deve ser cls ou mean, não  {parm_tipo_resumo}")

    # normalizando
    embeddings    = embeddings / torch.norm(embeddings, dim=1, keepdim=True)
    return embeddings.to('cpu').squeeze()
    # Retorna as embeddings geradas como um tensor do PyTorch


In [282]:
def gera_embedding_corpus(model, parm_tokenizer, corpus, parm_batch_size:int=64, parm_type:str='cls'):
    embeddings = None
    qtd_bloco = math.ceil(len(corpus)/parm_batch_size) 
    for ndx in tqdm(range(qtd_bloco)):
        # print(ndx, f"ndx*parm_batch_size:ndx*parm_batch_size+parm_batch_size {ndx}*{parm_batch_size}:{ndx}*{parm_batch_size}+{parm_batch_size} ")
        lista_doctos = [docto['title'] + ' ' + docto['text'] for docto in corpus[ndx*parm_batch_size:ndx*parm_batch_size+parm_batch_size]]
        embeddings_batch = gera_embeddings(parm_model=model, parm_tokenizer=parm_tokenizer, parm_sentences=lista_doctos, parm_tipo_resumo=parm_type)
        if embeddings is None:
            embeddings = embeddings_batch
        else:
            embeddings = torch.cat( (embeddings, embeddings_batch), dim=0)

        # embeddings.extend(embeddings_batch)
    print("embeddings gerados: shape {embeddings.shape}")
    return embeddings



In [283]:
def gera_embedding_queries(model, parm_queries, parm_type:str='cls'):
    dict_queries_encoded = {}
    for id, value in parm_queries.items():
        # print(id, value)
        dict_queries_encoded[id]= gera_embeddings(model, tokenizer, [value['question']], 'cls')
    return dict_queries_encoded

In [284]:
def run_all_queries_busca_exaustiva(parm_dict_queries_encoded:{}, parm_corpus_encoded, parm_lista_doc_id_passage:list, parm_num_max_hits:int=1000):
    # versão baseada na do colega Carísio, com adaptaçõess
    with open(CAMINHO_RUN, 'w') as runfile:
        for cnt, (query_id, query_embedding) in enumerate(parm_dict_queries_encoded.items()):
            if cnt % 5 == 0:
                print(f'{cnt} queries completadas')

            # Pega os primeiros 1000 resultados

            score = torch.matmul(parm_corpus_encoded, query_embedding)
            # Ordena
            sorted_score, indices_score = torch.sort(score, descending=True)

            # parm_num_max_hits primeiros
            sorted_score = sorted_score[0:parm_num_max_hits]
            indices_score = indices_score[0:parm_num_max_hits]

            # ids dos documentos
            ids_docs = [parm_lista_doc_id_passage[i] for i in indices_score]

            #  query  q0     docid  rank     score    descr    
            for i, (id_doc, score) in enumerate(zip(ids_docs, sorted_score)):
                texto_docto = f'{query_id} Q0 {id_doc} {i+1} {float(score):.6f} Pesquisa\n'
                _ = runfile.write(texto_docto)



                

In [ ]:
def get_classe_mais_proxima(vetor_query): 
  # fonte: colega Carísio
  classe_mais_proxima = -1
  dist_mais_proxima = 1e10
  for classe, vetor_classe in enumerate(kmeans.cluster_centers_):
    dist = np.linalg.norm(vetor_query - vetor_classe)
    if dist < dist_mais_proxima:
      dist_mais_proxima = dist
      classe_mais_proxima = classe
  
  return classe_mais_proxima


In [ ]:
def run_all_queries_busca_aproximada(parm_dict_queries_encoded:{}, 
            parm_corpus_encoded, 
            parm_lista_doc_id_passage:list,
            parm_num_max_hits:int=1000):
    # versão baseada na do colega Carísio, com adaptaçõess
    with open(CAMINHO_RUN_BUSCA_APROXIMADA, 'w') as runfile:
        for cnt, (query_id, query_embedding) in enumerate(parm_dict_queries_encoded.items()):
            if cnt % 5 == 0:
                print(f'{cnt} queries completadas')

            classe_mais_proxima = get_classe_mais_proxima(query_embedding)
            indices_na_classe = indices_por_cluster[classe_mais_proxima]['indice']

            matriz_filtrada = parm_corpus_encoded[torch.LongTensor(indices_na_classe).to(torch.int)]
            ids_docs_filtrados = indices_por_cluster[classe_mais_proxima]['ids_doc']
            
            # pesquisa_query_e_retorna_n_primeiros_docs(matriz_filtrada, ids_docs_filtrados, query, n)
            score = torch.matmul(matriz_filtrada, query_embedding)
            # Ordena
            sorted_score, indices_score = torch.sort(score, descending=True)
            # parm_num_max_hits primeiros
            sorted_score = sorted_score[0:parm_num_max_hits]
            indices_score = indices_score[0:parm_num_max_hits]
            # Extrai os ids dos documentos
            ids_docs = [ids_docs_filtrados[i] for i in indices_score]

            #  query  q0     docid  rank     score    descr    
            for i, (id_doc, score) in enumerate(zip(ids_docs, sorted_score)):
                texto_docto = f'{query_id} Q0 {id_doc} {i+1} {float(score):.6f} Pesquisa\n'
                _ = runfile.write(texto_docto)



                

# Pipeline com modelo que fiz fine-tuning

In [285]:
PATH_MODELO_FINAL = "/home/borela/fontes/deep_learning_em_buscas_unicamp/local/dpr/dpr_best_model_final"

In [286]:
# Se tiver que treinar os modelos, abre
models = {'query': AutoModel.from_pretrained(f"{PATH_MODELO_FINAL}_query").to(device),
'passage' : AutoModel.from_pretrained(f"{PATH_MODELO_FINAL}_passage").to(device)}
print(models['query'].config.max_position_embeddings, models['passage'].config.vocab_size)
for model in models:
    models[model].eval()
# 512 e 30522
#models['query'].config.__dict__


512 30522


In [287]:
tokenizer = AutoTokenizer.from_pretrained('microsoft/MiniLM-L12-H384-uncased')

### Experimentos de teste de encoding

In [288]:
sentences_test = ["This is an example sentence", "Each sentence is converted"]

In [289]:
# Tokenize sentences
encoded_input = tokenizer(sentences_test, padding=True, truncation=True, return_tensors='pt')
print(encoded_input)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


{'input_ids': tensor([[ 101, 2023, 2003, 2019, 2742, 6251,  102],
        [ 101, 2169, 6251, 2003, 4991,  102,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0]])}


In [290]:
encoded_input = tokenizer.pad(encoded_input, return_tensors='pt')
print('após padding',encoded_input)


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


após padding {'input_ids': tensor([[ 101, 2023, 2003, 2019, 2742, 6251,  102],
        [ 101, 2169, 6251, 2003, 4991,  102,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0]])}


In [291]:
# print(f"padded_batch {padded_batch}")
encoded_input = BatchEncoding(encoded_input)
print('após BatchEncoding',encoded_input)

após BatchEncoding {'input_ids': tensor([[ 101, 2023, 2003, 2019, 2742, 6251,  102],
        [ 101, 2169, 6251, 2003, 4991,  102,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0]])}


In [292]:
# Move os dados para o dispositivo especificado (CPU ou GPU)
encoded_input = {key: value.to(device) for key, value in encoded_input.items()}
print(encoded_input)

{'input_ids': tensor([[ 101, 2023, 2003, 2019, 2742, 6251,  102],
        [ 101, 2169, 6251, 2003, 4991,  102,    0]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0]], device='cuda:0')}


In [293]:
models['passage']

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 384, padding_idx=0)
    (position_embeddings): Embedding(512, 384)
    (token_type_embeddings): Embedding(2, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [294]:
# Compute token embeddings
with torch.no_grad():
    model_output = models['passage'](**encoded_input)

In [295]:
model_output.last_hidden_state.shape

torch.Size([2, 7, 384])

In [296]:
len(model_output)

2

In [297]:
embeddings = model_output.last_hidden_state[:, 0, :]
print(embeddings.shape)
print(embeddings[0,:10])

torch.Size([2, 384])
tensor([3.1420e-01, -1.6939e-01, -1.3007e-01, -2.6283e-01, 3.6589e-01, -1.5062e-01, -2.1353e-01, 1.5691e-02,
        4.9411e-01, 1.8600e-01], device='cuda:0')


In [298]:
embeddings    = embeddings / torch.norm(embeddings, dim=1, keepdim=True)
print(embeddings.shape)
print(embeddings[0,:10])

torch.Size([2, 384])
tensor([3.5197e-02, -1.8975e-02, -1.4571e-02, -2.9443e-02, 4.0988e-02, -1.6872e-02, -2.3920e-02, 1.7577e-03,
        5.5351e-02, 2.0836e-02], device='cuda:0')


## Geração dos embeddings 

### Gerando enconding para queries

In [312]:
dict_queries_encoded = gera_embedding_queries(models['query'], topics, parm_type='cls')

In [313]:
len(dict_queries_encoded), dict_queries_encoded[1][:10]

(50,
 tensor([-6.1646e-02, 4.6534e-02, -2.1565e-02, -2.2928e-02, 3.1714e-02, -4.7085e-02, -4.7286e-02, 1.2534e-02,
         -2.9679e-02, 2.3128e-02]))

### Gerando enconding para docs

sem batch: 25min
batch size 256: 10min,26s
batch size 512: 10min,15s

In [302]:
%%time
corpus_encoded = gera_embedding_corpus(models['passage'], parm_tokenizer=tokenizer, corpus=corpus, parm_batch_size=512, parm_type='cls')

100%|██████████| 335/335 [10:16<00:00,  1.84s/it]

embeddings gerados: shape {embeddings.shape}
CPU times: user 21min 23s, sys: 27.5 s, total: 21min 50s
Wall time: 10min 16s


In [307]:
corpus_encoded[0,:10]

tensor([-2.0054e-02, -2.2153e-02, 2.0548e-02, -5.3252e-03, 9.7119e-02, -2.8494e-02, 6.3460e-03, 5.7801e-02,
        -7.7143e-03, 4.6578e-03])

In [306]:
len(corpus_encoded)

171332

In [308]:
mostra_memoria(['cpu','gpu'])

Your runtime RAM in gb: 
 total 67.35
 available 55.52
 used 10.62
 free 51.8
 cached 4.55
 buffers 0.38
/nGPU
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Wed Apr 19 15:04:35 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.39.01    Driver Version: 510.39.01    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+====================

In [309]:
torch.cuda.empty_cache()

In [310]:
mostra_memoria(['cpu','gpu'])

Your runtime RAM in gb: 
 total 67.35
 available 55.49
 used 10.64
 free 51.77
 cached 4.56
 buffers 0.38
/nGPU
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Wed Apr 19 15:04:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.39.01    Driver Version: 510.39.01    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+===================

### Salvando os dados dos embeddings

In [314]:
CAMINHO_ARQUIVO_EMBEDDINGS = f"{DIRETORIO_TRABALHO}/data_index_meu_modelo_norm_cls.pickle"

In [315]:
with open(CAMINHO_ARQUIVO_EMBEDDINGS, 'wb') as outputFile:
    pickle.dump({'dict_queries_encoded': dict_queries_encoded,
                 'corpus_encoded': corpus_encoded}, outputFile, pickle.HIGHEST_PROTOCOL)

## Buscando os doctos para as queries - busca exaustiva


### Lendo os dados salvos

In [ ]:
with open(CAMINHO_ARQUIVO_EMBEDDINGS, "rb") as f:
  teste = pickle.load(f)

In [ ]:
dict_queries_encoded = teste['dict_queries_encoded']
corpus_encoded = teste['corpus_encoded']

In [316]:
assert len(corpus_encoded)==171332, f"Tamanho len(corpus_encoded) deveria ser 171332 e foi {len(corpus_encoded)}"
assert len(dict_queries_encoded)==50, f"Tamanho len(dict_queries_encoded) deveria ser 171332 e foi {len(dict_queries_encoded)}"

In [317]:
corpus_encoded.shape

torch.Size([171332, 384])

In [318]:
run_all_queries_busca_exaustiva(parm_dict_queries_encoded=dict_queries_encoded,
 parm_corpus_encoded=corpus_encoded, 
 parm_lista_doc_id_passage=lista_doc_id_passage, parm_num_max_hits=1000)

0 queries completadas
5 queries completadas
10 queries completadas
15 queries completadas
20 queries completadas
25 queries completadas
30 queries completadas
35 queries completadas
40 queries completadas
45 queries completadas


## Avaliação

In [319]:
### Calculando métricas
run = pd.read_csv(f"{CAMINHO_RUN}", sep="\s+", 
                names=["query", "q0", "docid", "rank", "score", "system"])
print(run.head())
run = run.to_dict(orient="list")
results = trec_eval.compute(predictions=[run], references=[qrel_dict])

# salvando métricas    
print(f"NDCG@10: {results['NDCG@10']}")
print(f"Resultados: {results}")


   query  q0     docid  rank     score    system
0     44  Q0  8t8psk46     1  0.883771  Pesquisa
1     44  Q0  6wxjm7m0     2  0.874808  Pesquisa
2     44  Q0  r1oqwdkz     3  0.874691  Pesquisa
3     44  Q0  2tmu1wzk     4  0.873972  Pesquisa
4     44  Q0  wjtc808r     5  0.870026  Pesquisa

[5 rows x 6 columns]
NDCG@10: 0.36520473545274407
Resultados: {'runid': 'Pesquisa', 'num_ret': 50000, 'num_rel': 24673, 'num_rel_ret': 4407, 'num_q': 50, 'map': 0.06862109415448163, 'gm_map': 0.019367495131862766, 'bpref': 0.17813768898671128, 'Rprec': 0.13289035688618756, 'recip_rank': 0.5784036042177612, 'P@5': 0.41999999999999993, 'P@10': 0.38199999999999995, 'P@15': 0.3693333333333333, 'P@20': 0.3670000000000001, 'P@30': 0.3413333333333333, 'P@100': 0.24660000000000004, 'P@200': 0.19270000000000004, 'P@500': 0.12732000000000002, 'P@1000': 0.08814, 'NDCG@5': 0.39433032879905805, 'NDCG@10': 0.36520473545274407, 'NDCG@15': 0.3524808170252889, 'NDCG@20': 0.348396848882813, 'NDCG@30': 0.3307442121

## Buscando os doctos para as queries - busca por clusters


In [320]:
from sklearn.cluster import KMeans

In [304]:
# Cálculo dos centróides:
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters, init='k-means++', max_iter=300, n_init=10, random_state=42)
classe_doc = kmeans.fit_predict(corpus_encoded)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [321]:
# separação dos índices dos documentos nesses 10 clusters e deixar em um dict só
# fonte: Colega Carísio
indices_por_cluster = {}

for classe in range(0, n_clusters):
  indices_por_cluster[classe] = {
      'indice': [indice for indice, classe_doc in enumerate(classe_doc) if classe == classe_doc],
      'ids_doc': [lista_doc_id_passage[indice] for indice, classe_doc in enumerate(classe_doc) if classe == classe_doc]
  }
  print(f'Total docs na classe {classe} = {len(indices_por_cluster[classe]["indice"])}')

print(indices_por_cluster[0]['indice'][0:3])
print(indices_por_cluster[0]['ids_doc'][0:3])

Total docs na classe 0 = 19370
Total docs na classe 1 = 23619
Total docs na classe 2 = 15997
Total docs na classe 3 = 12729
Total docs na classe 4 = 10077
Total docs na classe 5 = 27049
Total docs na classe 6 = 19107
Total docs na classe 7 = 12179
Total docs na classe 8 = 9085
Total docs na classe 9 = 22120
[128, 193, 197]
['mzn448zk', 'sswimukk', 'g370ygbu']


In [325]:

get_classe_mais_proxima(dict_queries_encoded[1])


3

In [331]:
run_all_queries_busca_aproximada(parm_dict_queries_encoded=dict_queries_encoded,
 parm_corpus_encoded=corpus_encoded, 
 parm_lista_doc_id_passage=lista_doc_id_passage, parm_num_max_hits=1000)

0 queries completadas
5 queries completadas
10 queries completadas
15 queries completadas
20 queries completadas
25 queries completadas
30 queries completadas
35 queries completadas
40 queries completadas
45 queries completadas


## Avaliação

In [332]:
### Calculando métricas
run = pd.read_csv(CAMINHO_RUN_BUSCA_APROXIMADA, sep="\s+", 
                names=["query", "q0", "docid", "rank", "score", "system"])
print(run.head())
run = run.to_dict(orient="list")
results = trec_eval.compute(predictions=[run], references=[qrel_dict])

# salvando métricas    
print(f"NDCG@10: {results['NDCG@10']}")
print(f"Resultados: {results}")


   query  q0     docid  rank     score    system
0     44  Q0  8t8psk46     1  0.883771  Pesquisa
1     44  Q0  wo9awq4g     2  0.868284  Pesquisa
2     44  Q0  a0mdq6xr     3  0.866033  Pesquisa
3     44  Q0  eimqd5b9     4  0.865012  Pesquisa
4     44  Q0  dc8j05vf     5  0.864485  Pesquisa

[5 rows x 6 columns]
NDCG@10: 0.31237215342977803
Resultados: {'runid': 'Pesquisa', 'num_ret': 50000, 'num_rel': 24673, 'num_rel_ret': 2721, 'num_q': 50, 'map': 0.034313994677037385, 'gm_map': 0.009811242772995174, 'bpref': 0.1104174652221865, 'Rprec': 0.08189459861574137, 'recip_rank': 0.5336316894693491, 'P@5': 0.36, 'P@10': 0.33, 'P@15': 0.3213333333333333, 'P@20': 0.30400000000000005, 'P@30': 0.27799999999999997, 'P@100': 0.18000000000000005, 'P@200': 0.1295, 'P@500': 0.08043999999999998, 'P@1000': 0.05442, 'NDCG@5': 0.3353950015646631, 'NDCG@10': 0.31237215342977803, 'NDCG@15': 0.3054478843497728, 'NDCG@20': 0.29059976725743564, 'NDCG@30': 0.2705672040529948, 'NDCG@100': 0.18758453135006886,

# Pipeline com modelo já treinado all-MiniLM-L12-v2

Para checar se seu codigo de avaliação está correto, comparar o seu desempenho com o do modelo já treinado no MS MARCO:   https://huggingface.co/sentence-transformers/all-MiniLM-L12-v2; O nDCG@10 no TREC-COVID deve ser ~0.47

In [ ]:
MODEL_NAME = 'sentence-transformers/all-MiniLM-L12-v2'

In [ ]:
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME).to(device)
model.eval()
#models['query'] = AutoModel.from_pretrained(MODEL_NAME).to(device)

models = {'passage': model,
    'query': model}

### Experimentos de teste de encoding

In [ ]:
sentences_test = ["This is an example sentence", "Each sentence is converted"]

In [ ]:
# Tokenize sentences
encoded_input = tokenizer(sentences_test, padding=True, truncation=True, return_tensors='pt')
print(encoded_input)

{'input_ids': tensor([[ 101, 2023, 2003, 2019, 2742, 6251,  102],
        [ 101, 2169, 6251, 2003, 4991,  102,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0]])}


In [ ]:
encoded_input = tokenizer.pad(encoded_input, return_tensors='pt')
print('após padding',encoded_input)


You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


após padding {'input_ids': tensor([[ 101, 2023, 2003, 2019, 2742, 6251,  102],
        [ 101, 2169, 6251, 2003, 4991,  102,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0]])}


In [ ]:
# print(f"padded_batch {padded_batch}")
encoded_input = BatchEncoding(encoded_input)
print('após BatchEncoding',encoded_input)

após BatchEncoding {'input_ids': tensor([[ 101, 2023, 2003, 2019, 2742, 6251,  102],
        [ 101, 2169, 6251, 2003, 4991,  102,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0]])}


In [ ]:
# Move os dados para o dispositivo especificado (CPU ou GPU)
encoded_input = {key: value.to(device) for key, value in encoded_input.items()}
print(encoded_input)

{'input_ids': tensor([[ 101, 2023, 2003, 2019, 2742, 6251,  102],
        [ 101, 2169, 6251, 2003, 4991,  102,    0]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0]], device='cuda:0')}


In [ ]:
models['passage']

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 384, padding_idx=0)
    (position_embeddings): Embedding(512, 384)
    (token_type_embeddings): Embedding(2, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [ ]:
# Compute token embeddings
with torch.no_grad():
    model_output = models['passage'](**encoded_input)

In [ ]:
model_output.last_hidden_state.shape

torch.Size([2, 7, 384])

In [ ]:
len(model_output)

2

In [ ]:
embeddings = model_output.last_hidden_state[:, 0, :]
print(embeddings.shape)
print(embeddings[0,:10])

torch.Size([2, 384])
tensor([5.1886e-02, -3.0087e-02, 2.1376e-01, 2.5196e-02, 9.8660e-02, -9.7260e-02, 8.8231e-02, 1.2464e-01,
        9.9461e-02, -4.3355e-02], device='cuda:0')


In [ ]:
embeddings    = embeddings / torch.norm(embeddings, dim=1, keepdim=True)
print(embeddings.shape)
print(embeddings[0,:10])

torch.Size([2, 384])
tensor([1.1514e-02, -6.6767e-03, 4.7438e-02, 5.5914e-03, 2.1894e-02, -2.1584e-02, 1.9580e-02, 2.7659e-02,
        2.2072e-02, -9.6212e-03], device='cuda:0')


## Geração dos embeddings 

### Gerando enconding para queries

In [ ]:
dict_queries_encoded = gera_embedding_queries(models['query'], topics, parm_type='cls')

In [ ]:
len(dict_queries_encoded), dict_queries_encoded[1][:10]

(50,
 tensor([-6.1646e-02, 4.6534e-02, -2.1565e-02, -2.2928e-02, 3.1714e-02, -4.7085e-02, -4.7286e-02, 1.2534e-02,
         -2.9679e-02, 2.3128e-02]))

### Gerando enconding para docs

pré-treinado 

        sem batch: 25min
        batch size 256: 10min,26s
        batch size 512: 10min,15s

        

In [ ]:
%%time
corpus_encoded = gera_embedding_corpus(models['passage'], parm_tokenizer=tokenizer, corpus=corpus, parm_batch_size=512, parm_type='cls')

In [ ]:
corpus_encoded[0,:10]

tensor([-6.4235e-05, -2.4650e-02, -5.4210e-02, -3.9027e-02,  1.7240e-02,
        -5.6386e-03, -4.2778e-02,  2.0824e-02, -7.0401e-03,  1.7235e-02])

In [ ]:
mostra_memoria(['cpu','gpu'])

Your runtime RAM in gb: 
 total 67.35
 available 58.68
 used 7.72
 free 56.04
 cached 3.32
 buffers 0.28
/nGPU
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Wed Apr 19 12:14:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.39.01    Driver Version: 510.39.01    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+====================

In [ ]:
torch.cuda.empty_cache()

In [ ]:
mostra_memoria(['cpu','gpu'])

Your runtime RAM in gb: 
 total 67.35
 available 58.67
 used 7.73
 free 56.03
 cached 3.31
 buffers 0.28
/nGPU
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Wed Apr 19 12:14:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.39.01    Driver Version: 510.39.01    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+====================

### Salvando os dados dos embeddings

In [ ]:
CAMINHO_ARQUIVO_EMBEDDINGS = f"{DIRETORIO_TRABALHO}/data_index_all_minilm_norm_cls.pickle"

In [ ]:
with open(CAMINHO_ARQUIVO_EMBEDDINGS, 'wb') as outputFile:
    pickle.dump({'dict_queries_encoded': dict_queries_encoded,
                 'corpus_encoded': corpus_encoded}, outputFile, pickle.HIGHEST_PROTOCOL)

## Buscando os doctos para as queries


### Lendo os dados salvos

In [ ]:
with open(CAMINHO_ARQUIVO_EMBEDDINGS, "rb") as f:
  teste = pickle.load(f)

In [ ]:
dict_queries_encoded = teste['dict_queries_encoded']
corpus_encoded = teste['corpus_encoded']

In [ ]:
assert len(corpus_encoded)==171332, f"Tamanho len(corpus_encoded) deveria ser 171332 e foi {len(corpus_encoded)}"
assert len(dict_queries_encoded)==50, f"Tamanho len(dict_queries_encoded) deveria ser 171332 e foi {len(dict_queries_encoded)}"

In [ ]:
corpus_encoded.shape

torch.Size([171332, 384])

In [ ]:
run_all_queries_busca_exaustiva(parm_dict_queries_encoded=dict_queries_encoded,
 parm_corpus_encoded=corpus_encoded, 
 parm_lista_doc_id_passage=lista_doc_id_passage, parm_num_max_hits=1000)

0 queries completadas
5 queries completadas
10 queries completadas
15 queries completadas
20 queries completadas
25 queries completadas
30 queries completadas
35 queries completadas
40 queries completadas
45 queries completadas


## Avaliação

In [ ]:
### Calculando métricas
run = pd.read_csv(f"{CAMINHO_RUN}", sep="\s+", 
                names=["query", "q0", "docid", "rank", "score", "system"])
print(run.head())
run = run.to_dict(orient="list")
results = trec_eval.compute(predictions=[run], references=[qrel_dict])

# salvando métricas    
print(f"NDCG@10: {results['NDCG@10']}")
print(f"Resultados: {results}")


   query  q0     docid  rank     score    system
0     44  Q0  a6gaoeie     1  0.950502  Pesquisa
1     44  Q0  appqx6sc     2  0.950065  Pesquisa
2     44  Q0  e3icgsl9     3  0.944932  Pesquisa
3     44  Q0  vtxu50wz     4  0.941929  Pesquisa
4     44  Q0  0zrlkbkx     5  0.940018  Pesquisa

[5 rows x 6 columns]
NDCG@10: 0.3697687091654517
Resultados: {'runid': 'Pesquisa', 'num_ret': 50000, 'num_rel': 24673, 'num_rel_ret': 4087, 'num_q': 50, 'map': 0.060542553645249964, 'gm_map': 0.013651706105066396, 'bpref': 0.1650159727700088, 'Rprec': 0.11628386262893219, 'recip_rank': 0.5874959521077168, 'P@5': 0.444, 'P@10': 0.38800000000000007, 'P@15': 0.3373333333333333, 'P@20': 0.314, 'P@30': 0.288, 'P@100': 0.19939999999999997, 'P@200': 0.15780000000000002, 'P@500': 0.11043999999999998, 'P@1000': 0.08174000000000001, 'NDCG@5': 0.40845529112965623, 'NDCG@10': 0.3697687091654517, 'NDCG@15': 0.3319511085204645, 'NDCG@20': 0.3135865419513995, 'NDCG@30': 0.2903264436958502, 'NDCG@100': 0.2103160

# Teste com índice Faiss que não deu certo (deixado para histórico e futuro ajuste)

#### Função para gerar índice usando faiss

In [ ]:
def gera_indice(parm_corpus_encoded:{}, parm_tipo_similaridade_indice:str='cosseno'):
    """
        Ainda não implementados outros tipos de similaridade.
        Outras métricas de similaridade suportadas pelo FAISS incluem a distância euclidiana (faiss.IndexFlatL2) e 
        a distância Manhattan (faiss.IndexFlatL1).
        Você pode consultar a documentação oficial do FAISS para obter mais informações sobre as métricas disponíveis e como usá-las: https://github.com/facebookresearch/faiss/blob/master/docs/indexes.md#available-indexes
    """
    embed_dim = parm_corpus_encoded.shape[1]  # dimensão dos embeddings
    print(f"embed_dim: {embed_dim}")    
    if parm_tipo_similaridade_indice == 'cosseno':
        #  a métrica cosseno é usada com o índice FlatIP, que é uma versão otimizada para a métrica cosseno do índice Flat, que é o índice padrão do FAISS.
        index = faiss.IndexFlatL2(embed_dim)  # índice com métrica L2 (euclidiana)
    else:
        raise Exception(f"parm_tipo_similaridade_indice deve ser cosseno, não  {parm_tipo_similaridade_indice}")
    index.add(parm_corpus_encoded.numpy())  # adiciona os embeddings do corpus ao índice
    return index

In [ ]:
%%time
indice  = gera_indice(corpus_encoded, 'cosseno')


embed_dim: 384
CPU times: user 54.5 ms, sys: 56.1 ms, total: 111 ms
Wall time: 260 ms


In [ ]:
type(indice)

faiss.swigfaiss_avx2.IndexFlatL2

In [ ]:
lista_doc_id_passage[:4]

['ug7v899j', '02tnwd4m', 'ejv2xln0', '2b73a28n']

In [ ]:
query_id =  10
query_value = dict_queries_encoded[10]


In [ ]:
query_value.unsqueeze(dim=0).shape

torch.Size([1, 384])

In [ ]:
scores_result_search, index_result_search = indice.search(query_value.unsqueeze(dim=0), 1000)  # realiza a pesquisa no índice


In [ ]:
scores_result_search.shape, index_result_search.shape

((1, 1000), (1, 1000))

In [ ]:
scores_result_search.squeeze().shape

(1000,)

Testando ordenação

In [ ]:
a = np.array([[0.11012619, 0.08140977, 0.08162256, 0.08971895,  0.12045887, 0.12120931, 0.13146943, 0.13230261, 0.13461232, 0.13461915]], dtype=float)
b = np.array([[58763, 113883, 128603, 127203, 58215, 37539, 97725, 167823, 169760, 39305]])

In [ ]:
# indices = a.argsort()[::-1]  # obtém os índices em ordem decrescente
indices = np.argsort(-a)[0]
print(indices)

[9 8 7 6 5 4 0 3 2 1]


In [ ]:
a = a[0][indices]
b = b[0][indices]

print(a)
print(b)


[0.13461915 0.13461232 0.13230261 0.13146943 0.12120931 0.12045887
 0.11012619 0.08971895 0.08162256 0.08140977]
[ 39305 169760 167823  97725  37539  58215  58763 127203 128603 113883]


In [ ]:
scores_result_search[0,:10], index_result_search[0,:10]

(array([0.08140977, 0.08162256, 0.08971895, 0.11012619, 0.12045887,
        0.12120931, 0.13146943, 0.13230261, 0.13461232, 0.13461915],
       dtype=float32),
 array([ 58763, 113883, 128603, 127203,  58215,  37539,  97725, 167823,
        169760,  39305]))

In [ ]:
lista_doc_id_passage[0]

'ug7v899j'

In [ ]:
order = np.argsort(scores_result_search[0])

## Realizar buscas 

In [ ]:
# Run all queries in topics, retrive top 1k for each query
def run_all_queries_embed_index_faiss(parm_dict_queries_encoded:{}, parm_indice_com_embeddings:faiss.swigfaiss_avx2.IndexFlatL2, parm_lista_doc_id_passage:list, parm_num_max_hits:int=1000):
    """
    A função run_all_queries é responsável por realizar todas as consultas armazenadas no dicionário topics utilizando o objeto searcher fornecido e salvar os resultados em um arquivo de texto.
    Usada no notebook da aula 2

    Parâmetros:

    file: caminho do arquivo de saída onde serão salvos os resultados das consultas.
    topics: dicionário contendo as consultas a serem executadas. Cada consulta é representada por uma chave única no dicionário. O valor correspondente a cada chave é um outro dicionário contendo as informações da consulta, como seu título e outras informações relevantes.
    searcher: objeto do tipo Searcher que será utilizado para realizar as consultas.
    num_max_hits: número máximo de documentos relevantes que serão retornados para cada consulta.
    Retorno:

    A função não retorna nenhum valor, mas salva os resultados das consultas no arquivo especificado em file.
    Comentário:

    A função usa a biblioteca tqdm para exibir uma barra de progresso enquanto executa as consultas.
    O número de consultas concluídas é impresso a cada 100 consultas.
    """
    global CAMINHO_RUN
    queries_encoded_np = np.array(list(parm_dict_queries_encoded.values()))
    scores_doctos_result_search = {}
    index_doctos_result_search = {}
    for query_id, query_value in parm_dict_queries_encoded.items():
        scores_doctos, index_doctos = indice.search(query_value.unsqueeze(dim=0), parm_num_max_hits)  # realiza a pesquisa no índice
        # print(f"query_id: {query_id}")
        # print(f"scores_doctos: {scores_doctos}")
        # print(f"index_doctos: {index_doctos}")
        # reordena por ordem descrescente de score
        indices = np.argsort(-scores_doctos)[0]
        scores_doctos_result_search[query_id] = scores_doctos[0][indices]
        index_doctos_result_search[query_id] = index_doctos[0][indices] 
    
    print("para query_id = 1")
    print(f"Após pesquisa, scores_doctos_result_search[1].shape: {scores_doctos_result_search[1].shape}, scores_doctos_result_search[1][:10]:{scores_doctos_result_search[1][:10]} ")
    print(f"Após pesquisa, index_doctos_result_search[1].shape: {index_doctos_result_search[1].shape} , index_doctos_result_search[1][:10]:{index_doctos_result_search[1][:10]} ")


    #  query  q0     docid  rank     score    descr
    with open(CAMINHO_RUN, 'w') as runfile:
        for query_id in parm_dict_queries_encoded.keys():
            # print(query_id)
            for i in range(0, parm_num_max_hits):
                ndx_docto = index_doctos_result_search[query_id][i]
                texto_docto = f'{query_id} Q0 {parm_lista_doc_id_passage[ndx_docto]} {i+1} {scores_doctos_result_search[query_id][i]:.6f} Pesquisa\n'
                _ = runfile.write(texto_docto)
                


In [ ]:
run_all_queries_embed_index_faiss(parm_dict_queries_encoded=dict_queries_encoded,
 parm_indice_com_embeddings=indice, 
 parm_lista_doc_id_passage=lista_doc_id_passage, parm_num_max_hits=1000)

/tmp/ipykernel_3312/3025828365.py:22: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  queries_encoded_np = np.array(list(parm_dict_queries_encoded.values()))
/tmp/ipykernel_3312/3025828365.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  queries_encoded_np = np.array(list(parm_dict_queries_encoded.values()))


para query_id = 1
Após pesquisa, scores_doctos_result_search[1].shape: (1000,), scores_doctos_result_search[1][:10]:[0.2639407  0.26384005 0.26374525 0.26370186 0.26363018 0.26363018
 0.2635759  0.2635759  0.26354355 0.2634717 ] 
Após pesquisa, index_doctos_result_search[1].shape: (1000,) , index_doctos_result_search[1][:10]:[ 88278  46571 110917 115432  99316 168440 112248 149914  60352 106858] 


## Avaliação dos resultados

Com cls

In [ ]:
trec_eval = load("trec_eval")

In [ ]:
### Calculando métricas
run = pd.read_csv(f"{CAMINHO_RUN}", sep="\s+", 
                names=["query", "q0", "docid", "rank", "score", "system"])
print(run.head())
run = run.to_dict(orient="list")
results = trec_eval.compute(predictions=[run], references=[qrel_dict])

# salvando métricas    
print(f"NDCG@10: {results['NDCG@10']}")
print(f"Resultados: {results}")


   query  q0     docid  rank     score    system
0     44  Q0  i00q5hfu     1  0.442576  Pesquisa
1     44  Q0  s8tg24te     2  0.442530  Pesquisa
2     44  Q0  4l446lug     3  0.442494  Pesquisa
3     44  Q0  fis7kq3b     4  0.442468  Pesquisa
4     44  Q0  62f8a96g     5  0.442273  Pesquisa

[5 rows x 6 columns]
NDCG@10: 0.040467891064239
Resultados: {'runid': 'Pesquisa', 'num_ret': 50000, 'num_rel': 24673, 'num_rel_ret': 4087, 'num_q': 50, 'map': 0.021051398306511784, 'gm_map': 0.00295207512878423, 'bpref': 0.16256979096245014, 'Rprec': 0.0564165619124504, 'recip_rank': 0.10332331368454076, 'P@5': 0.044000000000000004, 'P@10': 0.05, 'P@15': 0.04800000000000001, 'P@20': 0.049, 'P@30': 0.04666666666666666, 'P@100': 0.043, 'P@200': 0.04699999999999999, 'P@500': 0.05304, 'P@1000': 0.08174000000000001, 'NDCG@5': 0.03598953016807816, 'NDCG@10': 0.040467891064239, 'NDCG@15': 0.03994276405299504, 'NDCG@20': 0.040997985599810924, 'NDCG@30': 0.03936349909230019, 'NDCG@100': 0.0368829476340253